## 3) Multiple Regression Analysis

- 두 개 이상의 연속형 독립변수가 연속형 종속변수에는 미치는 영향을 검증하는 분석 방법

- 독립변수 분석: 종속변수와 관련이 있는 주요 독립변수 파악

### 격자 Data  전처리

In [170]:
# 격자데이터 읍면동별로 분류
df = gpd.sjoin(geo, data32, op='intersects', how='left')
dff = df.groupby('id').count().reset_index()
fine = dff[dff['left'] == 1][['id','left']]#하나씩으로 정상적으로 나눠진 데이터
fine = pd.merge(fine, df, on='id')[['id','ADM_DR_NM','left_x']]
dff = dff[dff['left']>1][['id','left']] #두개 이상이 할당된 데이터
df2 = pd.merge(dff, df, on='id')[['id','ADM_DR_NM','left_x']]

# 조치원 부분만 처리
eup = df2[df2['ADM_DR_NM'].str[3] == '읍']
a = eup['id'].unique().tolist()
eup = df2[df2['id'].isin(a)]
df2 = df2.drop(eup.index)
eup = eup[eup['ADM_DR_NM'].str[3] == '읍']


# 중복되는 읍면동 drop
dropdu = df2[['id','left_x']].drop_duplicates().index
df2 = df2.loc[dropdu,:]
done = pd.concat([fine, df2, eup])
emdgeo = pd.merge(done,geo, on='id')

In [171]:
# 읍면동 지역으로 나누어서 그리드 확인
eup = emdgeo[emdgeo['ADM_DR_NM'].str[3] == '읍']
myun = emdgeo[emdgeo['ADM_DR_NM'].str[2] == '면']
dong = emdgeo[emdgeo['ADM_DR_NM'].str[2] == '동']

print('읍: ', len(eup))
print('면: ', len(myun))
print('동: ', len(dong))

읍:  123
면:  2911
동:  149


In [172]:
# 지역구분 column 추가
eup['지역 구분'] = '읍'
myun['지역 구분'] = '면'
dong['지역 구분'] = '동'
df = pd.concat([eup, myun, dong])

### 지수(생활 편의, 안전, 교통), 공시지가, 인구 데이터 Column으로 추가

In [173]:
life_rate.columns = ['life','id']
safety_rate.columns = ['safety','id']
transport_rate.columns = ['transport','id']
gs = gongsi2[['id',2020,'변동률']]
gs.columns = ['id','공시지가','공시지가 변동률']
pop2020 = pop2020[['id','총합']].fillna(0).replace(0,1)

In [174]:
popin = popin[['index',0]].fillna(0)
popin.columns = ['ADM_DR_NM','순전입자수']
# 순전입자수가 너무 많기 때문에 scaling 진행
min_max_scaler = MinMaxScaler(feature_range=(5,100))
x=popin['순전입자수']
x=x.to_numpy(dtype=float)
x=x.reshape(-1,1)
x1= min_max_scaler.fit_transform(x)
popin['순전입자수'] = x1

In [175]:
age = age.replace(0,1)

In [176]:
from functools import reduce
dfs = [df,life_rate,safety_rate,transport_rate, gs, age, pop2020]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), dfs, )
df_merge = pd.merge(df_merge, popin, on='ADM_DR_NM')
emd = df_merge.drop(['left_x','left','top','bottom','SIG_CD','SIG_KOR_NM'],axis=1)

In [177]:
eup = emd[emd['지역 구분'] == '읍']
myun = emd[emd['지역 구분'] == '면']
dong = emd[emd['지역 구분'] == '동']

### 다중선형회귀( Level-log Model) 함수 정의

- x가 1%만큼 증가할 때 y가 $\frac{\beta}{100}$% 만큼 변하는 것을 의미하는 분석 방법 ➡︎ 독립변수를 단위화  
  
- 독립변수가 단위에 영향을 받지 않도록 행복지수, 안전지수, 교통지수에 대해서 Level-log Model을 사용

In [178]:
from statsmodels.formula.api import ols

def log(df):
    # 공시지가 na 값 drop 후, na 아닌 데이터만 추출
    df=df.reset_index()
    df1=df['공시지가'].dropna()
    index=df1.index
    df=df.loc[index]
    df = df.fillna(0)

    # 로그 씌워주기
    df['log생활지수']=np.log(df['life'])
    df['log안전지수']=np.log(df['safety'])
    df['log교통지수']=np.log(df['transport'])
    df['log건축연한']=np.log(df['사용연도'])
    df['건축연한']=df['사용연도']
    df['log순전입자수']=np.log(df['순전입자수'])

    return df

def ourols(df, log=False):
    # 회귀 돌려주기
    res = ols('공시지가 ~ log생활지수+log안전지수+log교통지수+순전입자수', data=df).fit()
        
    return res

### 읍 (log-level)

In [179]:
res1 = ourols(log(eup))
res1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   공시지가   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.440
Method:                 Least Squares   F-statistic:                     22.45
Date:                Fri, 26 Mar 2021   Prob (F-statistic):           1.31e-10
Time:                        22:57:53   Log-Likelihood:                -430.30
No. Observations:                  83   AIC:                             868.6
Df Residuals:                      79   BIC:                             878.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4302      0.525     -0.819      0.415      -1.476       0.615
log생활지수       33.3251     12.429      2.681      0.009       8.586      58.065
log안전지수       26.7721      8.618      3.106      0.003       9.618      43.927
log교통지수       17.8212     10.543      1.690      0.095      -3.165      38.807
순전입자수         -2.7477      3.355     -0.819      0.415      -9.425       3.929
==============================================================================
Omnibus:                       11.133   Durbin-Watson:                   1.394
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.567
Skew:                           0.771   Prob(JB):                      0.00308
Kurtosis:                       3.984   Cond. No.                     2.41e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.89e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 면 (log-level)

In [180]:
res2 = ourols(log(myun))
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   공시지가   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     245.6
Date:                Fri, 26 Mar 2021   Prob (F-statistic):          2.24e-159
Time:                        22:57:53   Log-Likelihood:                -7209.0
No. Observations:                1386   AIC:                         1.443e+04
Df Residuals:                    1381   BIC:                         1.445e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -369.9390     17.730    -20.865      0.000    -404.720    -335.158
log생활지수      147.1045     11.560     12.725      0.000     124.427     169.782
log안전지수       42.9230      3.506     12.242      0.000      36.045      49.801
log교통지수       48.0385      7.071      6.793      0.000      34.167      61.910
순전입자수          5.6526      0.653      8.656      0.000       4.372       6.934
==============================================================================
Omnibus:                      342.545   Durbin-Watson:                   1.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1865.500
Skew:                           1.038   Prob(JB):                         0.00
Kurtosis:                       8.291   Cond. No.                         145.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 동 (log-level)

In [181]:
res3 = ourols(log(dong))
res3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   공시지가   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     3.592
Date:                Fri, 26 Mar 2021   Prob (F-statistic):             0.0118
Time:                        22:57:54   Log-Likelihood:                -395.93
No. Observations:                  56   AIC:                             801.9
Df Residuals:                      51   BIC:                             812.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -584.2223    305.513     -1.912      0.061   -1197.565      29.120
log생활지수       81.2053     67.082      1.211      0.232     -53.467     215.877
log안전지수      173.4829     92.579      1.874      0.067     -12.377     359.342
log교통지수       45.1004     52.157      0.865      0.391     -59.609     149.810
순전입자수          3.6157      1.484      2.437      0.018       0.637       6.594
==============================================================================
Omnibus:                       13.538   Durbin-Watson:                   1.641
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.520
Skew:                           1.086   Prob(JB):                     0.000703
Kurtosis:                       4.228   Cond. No.                         422.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""